# <center> NSF REU 2024- Introduction to Python Day 4 </center>
### <center>  written by Jackie Champagne (University of Arizona) , Oscar Chavez Ortiz and Kay Guo (UT Austin)  </center>

# Day4: Astropy, FITS files & Imaging

A package I highly recommend you become familiar with is **astropy**. This includes a library of astronomical constants and unit conversions, coordinate conversions, cosmological calculations, image processing, and much more! There are many components of astropy that we won't be able to cover, but as a sneak peak you can deal with tables of data, measuring photometry, and processing psfs in images. See more here: https://docs.astropy.org/en/stable/index.htm


In [ ]:
import astropy
astropy.__version__


## Astropy: Units & Constants

The most basic use for Astropy is **units and constants.** I highly recommend using the constants within Astropy so that a) you're not constantly Googling them, and b) The constants are loaded with units as well, making conversions between them quite easy.

*One quick disclaimer is that the solar mass constant is slightly out of date, so researchers doing high-precision stellar work may need to keep that in mind.*

Load up the constants as well as our other usual packages:

#### Constants

In [ ]:
# import the consts
# Only import certain packages
from astropy import constants as const

import numpy as np

import matplotlib.pyplot as plt

To call a constant, simply type const.(constant) where the symbols are generally the same as what's in your textbooks. For example, G is Newton's gravitational constant:

In [ ]:
G = const.G

#print(G)
#const.G

#const.G.unit

Calling the constant will give you a bunch of information about it, including its name, value, uncertainty, and SI unit. Astropy is not in cgs units, but it's easy to convert by adding .cgs to the end of the unit:

In [ ]:
G_cgs = const.G.cgs

print(G_cgs)

This can be very helpful when using normalized equations. **Note that Python will have trouble with certain units: for example, it doesn't know it can cancel out Hz and seconds. Also, if there are units inside a log expression, it will complain.** So if you already know your units work out and you just want the value, you can type .value!

These can also be combined, like G.cgs.value.

In [ ]:
G_value = const.G.value

print('G_value is ', G_value, '\nG_value has the type of', type(G_value))

In [ ]:
print('h', const.h,'\n')

print('c', const.c,'\n')

print('M_sun', const.M_sun,'\n')

#### Units
When doing analysis, we have to do deal with datasets that have different units. It's always helpful to add a unit to your data to reduce calculation errors. Here is the link to astropy.units : https://docs.astropy.org/en/stable/units/index.html


In [ ]:
from astropy import units as u

R = 1.0 * u.kpc

print(R)


In [ ]:
#It can be more complex units
a = 3* u.km*u.s**2

In [ ]:
# units can be added to an array
flux_arr = np.array([1,2,3,4,5,6,7])
flux_arr_withu = flux_arr*u.nJy
print(flux_arr_withu)
print(flux_arr_withu[3])

### The powerful **.to()** 
    Unit conversion

In [ ]:
# G in kpc3/Msun Gyr2 is used a lot in astronomy
G_weirdunits = const.G.to('kpc3 / (Msun Gyr2)')

print(G_weirdunits)

#you can aslo do this in the other way
G_weirdunits_test = const.G.to(u.kpc**3/(u.Msun*u.Gyr**2))

In [ ]:
# We define  flux_arr_withu , let's change the units for the whole array
flux_arr_withu.to(u.Jy)


### Exercise

We know stefan-Boltzman law:
    $L = 4\sigma_{sb}$$\pi$$R^2T^4$, now you have R = 700000km, T = 6000K, please give me the L value in units : L_sun (luminosity of the sun)
    You can find the constants here : https://docs.astropy.org/en/stable/constants/index.html
    

## Astropy: Cosmological Calculations

There's a good chance you will need to calculate things like luminosity distance, angular diameter distance, or age of the Universe, especially if you are doing high-redshift work. To do cosmology in astropy, first set the cosmological parameters, like so:

In [ ]:
from astropy.cosmology import FlatLambdaCDM

cosmo=FlatLambdaCDM(H0=70., Om0=0.3)

This sets the Hubble constant to 70 km/s / Mpc and the matter density factor, $\Omega_M$, to 0.3. The cosmological calculations are attributes of FlatLambdaCDM which we've set to "cosmo." Here are a couple of examples:

In [ ]:
#latest version
from astropy.cosmology import Planck18 as cosmo

#put redshfit and it will convert redshift to distance 

LD4 = cosmo.luminosity_distance(4)#.to('kpc').value
ADD4 = cosmo.angular_diameter_distance(4)

Age4 = cosmo.age(4)

print(Age4)
print(LD4.to('kpc').value)

#### Exercise 
    Find out what is the age of the universe with Planck18
    Masie's galaxy is about redshift of 12, find out what is the luminosity_distance of it, and convert that into units of km.

## Astropy: Sky Coordinate Systems

Another fundamental part of astronomy is celestial coordinate systems. You will quickly learn that there are many ways to present the locations of objects in the sky: through right ascension and declination vs. galactic longitude and latitude for instance, or in degrees vs. hourangle, or in different frames of reference. With astropy.coordinates it is easy to convert between these.

In [ ]:
from astropy.coordinates import SkyCoord

#Say you want to define a star at 150.00 degrees, +2.00 degrees

#either would work
coords = SkyCoord(ra=150.0, dec=2.00, unit='deg', frame='icrs')
coords_2 = SkyCoord(ra=150.0*u.deg, dec=2.00*u.deg)



coords.ra

Now we can access other information or convert it to another format.

In [ ]:
print(coords.ra.hms)
print(coords.dec.degree)
print(coords.to_string('hmsdms'))

# Can convert to galactic coordinates l, b
coords_galactic = coords.galactic
print(coords_galactic)

# Matches and Separations

SkyCoord objects have some pretty awesome attributes that allow you to match between two different catalogs and gives a quick and easy way to compute the separation between different objects. This comes in handy when you need to find an imaging counterpart for an emission line, or when you are checking the astrometry (coordinates) of an image.

For more information click the link here: https://docs.astropy.org/en/stable/coordinates/matchsep.html

The first thing we will cover is separation. It takes two Sky Coordinates objects and it will give you back the separation between them.

In [ ]:
coords = SkyCoord(ra=150.0, dec=2.00, unit='deg', frame='icrs')
counterpart = SkyCoord(ra=150.000056, dec=2.000043, unit='deg', frame='icrs')

sep = coords.separation(counterpart)

print(f'The separation of the counterpart is: {sep}')
print(f'The separation of the counterpart is: {sep.to(u.arcsec)}')

In [ ]:
#You can also change the units of the separation with ease using the following

#give separation in arcseconds
print(f'Separation in arcseconds is: {sep.to(u.arcsec):.3f}')

#give separation in arcminutes
print(f'Separation in arcminutes is: {sep.to(u.arcmin):.5f}')

#gives separation in degrees
print(f'Separation in degrees is: {sep.to(u.deg):.6}')

#gives separation in radian
print(f'Separation in radians is: {sep.to(u.radian):.8f}')

In [ ]:
#You can also find separation between an array of SkyCoordinates

skycoord_arrays = SkyCoord(ra = np.random.uniform(100, 200, 10000) * u.degree, 
                           dec = np.random.uniform(0, 10, 10000) * u.degree)

sep = coords.separation(skycoord_arrays).to(u.arcsec)


print(sep)

In [ ]:
#you can then impose conditions to find candidate counterparts by using a search radius

#makes a boolean mask for us to then apply to sep variable to filter only the 
#sources that satisfy this condition
possible_match_mask = sep.degree < 1 #searching around a 1 degree search radius

print('Matches in skycoords array within 1 degree of coords:')
skycoord_arrays[possible_match_mask]

##  Exercise:
    Read the Final_Sample.txt from yestersday's Seminar
    Read the RA/DEC  (they are in units of degree)
    Calculate the seperration bewteen the first two sources: source index = 112 and source index = 145.
    Show the seperation in units of arcsec
    


# Matching Catalogs

The next very useful attribute of skycoords is its ability to quickly find nearby matches between two different catalogs. 

In [ ]:
import pandas as pd

spectra_df = pd.read_csv('NEP_Spectra_Sources_Coordinates.txt', sep = ' ', index_col = 0)

photom_df = pd.read_csv('NEP_Photom_Sources_Coordinates.txt', sep = ' ', index_col = 0)

spec_ra, spec_dec = spectra_df['ra'].values, spectra_df['dec'].values

photom_ra, photom_dec = photom_df['ALPHA_DETECTION'].values, photom_df['DELTA_DETECTION'].values

In [ ]:
catalog1 = SkyCoord(ra = spec_ra * u.degree, 
                    dec = spec_dec * u.degree)

catalog2 = SkyCoord(ra = photom_ra * u.degree, 
                    dec = photom_dec * u.degree)

#We try to find matches *to* catalog 1 *from* catalog2, that is we are finding closest match from catalog2 to catalog1
idx, sep2d, _ = catalog1.match_to_catalog_sky(catalog2)


#idx is the index in catalog 2 that was the closest match to the coordinates in catalog1
#sep2d is the separation between the closest match, defaults to degrees
#sep3d the 3d separation only useful if you have distance in SkyCoord, not useful here

matches_to_cat1 = catalog2[idx]

close_matches_mask = sep2d.arcsec < 0.5 # checking which coordinates are w/in 0.5 arcsec of one another

#applying mask to catalog 1
print('Matches in Catalog 1 within 0.5 arcsec: ')
print(catalog1[close_matches_mask])
print()

#applying mask to matches in catalog2
print('Matches in Catalog 2 within 0.5 arcsec: ')
print(matches_to_cat1[close_matches_mask])
print()

print(catalog1[close_matches_mask].separation(matches_to_cat1[close_matches_mask]).deg)

In [ ]:
#quick check how things worked
fig, ax = plt.subplots(1,2, figsize =(10,5), sharey = True)

ax[0].scatter(catalog1.ra, catalog1.dec, s =1, marker = ',', color = 'red', label = 'Cat1', zorder = 20)
ax[0].scatter(catalog2.ra, catalog2.dec, s =1, marker = ',', color = 'blue', label = 'Cat2')
#ax[0].legend()


ax[1].scatter(catalog1.ra[close_matches_mask], catalog1.dec[close_matches_mask], s =0.5, color = 'red', label = 'Cat1')
ax[1].scatter(matches_to_cat1.ra[close_matches_mask], matches_to_cat1.dec[close_matches_mask], s =0.5, color = 'blue', label = 'Cat2',zorder = -10)
#ax[1].legend()

ax[1].set_xlabel('RA (deg)')
ax[0].set_xlabel('RA (deg)')

ax[0].set_ylabel('DEC (deg)')

plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.0,
                    wspace=0.0)

# Astropy Tables

The next major thing to know about is using astropy tables. These tables will be some of the data that may be stored in FITS files which we will cover in the next section. This section will teach you how to make an astropy table, and append values to it. 

In [ ]:
from astropy.table import QTable, Table, Column, vstack, hstack, join

In [ ]:
#Creating a Table Numero 1

#creates an empty table
t = Table()

#fills in the table with data with the columns being 'a', 'b', and 'c'
# with the corresponding data for each column
t['a'] = [1, 4]
t['b'] = [2.0, 5.0]
t['c'] = ['x', 'y']

In [ ]:
#Creating a Table Numero 2

#we make our three columns into arrays
a = np.array([1, 4], dtype=np.int32)
b = [2.0, 5.0]
c = ['x', 'y']

#generate the table below with the arrays as the columns and the names of the columns
#in the names 
t = Table([a, b, c], names=('a', 'b', 'c'))

In [ ]:
#Creating a Table Numero 3

#this is using a mehtod called dictionaries which are a type of containers 
#they use a key to access data with the first entry the key and the next value the data
#we can see in this one that the keys become the columns names and that the data becomes
#the data in the columns
arr = {'a': np.array([1, 4], dtype=np.int32),
        'b': [2.0, 5.0],
        'c': ['x', 'y']}

Table(arr)

There is no right or wrong way to generate an astropy table, it all comes down to personal preference and what data you are handling. Sometimes you do not know what data you will store so method 1 may be best, othertimes you may have data in the forms of dictionaries and so method 3 would work best here. 

# Getting values from the astropy table

In [ ]:
arr = np.arange(15).reshape(5, 3)
t = Table(arr, names=('a', 'b', 'c'), meta={'keywords': {'key1': 'val1'}})

In [ ]:
t['a']       # Column 'a'
t['a'][1]    # Row 1 of column 'a'
t[1]         # Row 1
t[1]['a']    # Column 'a' of row 1
t[2:5]       # Table object with rows 2:5
t[[1, 3, 4]]  # Table object with rows 1, 3, 4 (copy)
t[np.array([1, 3, 4])]  # Table object with rows 1, 3, 4 (copy)
t[[]]        # Same table definition but with no rows of data
t['a', 'c']  # Table with cols 'a', 'c' (copy)
dat = np.array(t)  # Copy table data to numpy structured array object
t['a'].quantity  # an astropy.units.Quantity for Column 'a'
t['a'] = t['a']*u.m
t['a'].to('km')  # an astropy.units.Quantity for Column 'a' in units of kilometers
t.columns[1]  # Column 1 (which is the 'b' column)
t.columns[0:2]  # New table with columns 0 and 1

### Most of the times, you don't need to create a table yourself, rather, you will need to read an existing table

In [ ]:
#let's read the Stefanon+2017 catalog

EGS_mass = Table.read('hlsp_candels_hst_wfc3_egs_v1_mass_cat.fits')

EGS_mass.columns

if 'ID' in list(EGS_mass.columns):
    EGS_mass.rename_column('ID','c_ID')

In [ ]:
# One I find really useful is to select from astropy table

def make_cut(catalog, mass_key ,redshift_key):
    
    """make cut of the catalog wrt mass and redshift
    catalog: astropy Table
    mass_key: 'string', name of the stellar mass column
    redshift_key: 'string', name of the redshitf
    
    """
    
    #change this below for desired cuts
    mass_lim = 9
    z_low =3
    z_high = 3.5
    
    #selection
    sel_mass = (catalog[mass_key]>=mass_lim)
    sel_z = ((catalog[redshift_key]>=z_low) & (catalog[redshift_key]<=z_high))
    
    #combine selection
    sel = sel_mass &sel_z
    
    print('Using CANDELS catalog, z range',z_low, ' to ' , z_high, ', lmass gt than ' , mass_lim)
    
    return sel


catalog = EGS_mass
mass_key ,redshift_key = 'M_neb_med', 'zbest'

sel = make_cut(catalog, mass_key ,redshift_key)

In [ ]:
EGS_mass_selection = EGS_mass[sel]


### Exercise : 
    Select everything that has zbest > 3 and <=3.3
    

### Join

In [ ]:
import astropy.io.ascii as Ascii
cat_EGS ='./' #change this to your own directory
EGS_sfr = Ascii.read(cat_EGS+'hlsp_candels_hst_wfc3_egs-barro19_multi_v1_sfr-egs-cat.txt')

#This is an astropy table object
EGS_sfr.rename_column('ID','c_ID')


In [ ]:
output_table =  join(EGS_mass_selection, EGS_sfr, keys = 'c_ID')
#output_table =  join(EGS_mass_selection, EGS_sfr['c_ID','SFR-ladder_total'], keys = 'c_ID')


In [ ]:
#This suppose to be an interactive session, but it failed to work on my labtop
output_table[0:10].show_in_notebook()


In [ ]:
# write the table to a file
output_table.write('test.fits', overwrite = True)

In [ ]:
output_table.columns

### Excersize
        1. Read the table you just saved
        2. rename 'SFR-ladder_total' to 'sfr_tot'
        3. Read 'Example_catalog_with_zspec_no_ID.fits'
        4. match the sources in Example_catalog_with_zspec_no_ID.fits with sources the table you just saved, the seperation is < 0.25 arcsec
        5. to confirm you did this correctly, make a plot or give me some stats (e.g., you can show the difference in RA or in Dec is on average every small)
    
    

# FITS Files

Astropy can read in FITS files, which is a typical format of astronomical images. FITS stands for Flexible Image Transport System, and most basic image viewers can't do anything with them since they aren't files like JPGs or GIFs. There are a few programs that will read them, including CASA if you're a radio astronomer, or DS9.

FITS is a useful format for astronomical data because it contains a lot of behind the scenes information. The header in particular will usually give you information about the telescope the data is from, the reference position in the sky for the data, the pixel scale, the size of the image, and more! First, let's import the package.

In [ ]:
from astropy.io import fits

Let's step through how to work with the files I've given you to work with. First, open the file using fits.open(). To get some basic information about the file, type fitsfile.info().

In [ ]:
hdulist = fits.open('f001a066.fits') #B filter image of Andromeda
hdulist.info()

The actual information of interest is contained in the header. FITS files can contain many images in the same file, but this is a 2D image, so it only contains one slice. We will still have to specify that it's the first slice, so we call "hdulist[0]". Note that you can do a shortcut in the first place and place the [0] at the end of fits.open(). 

Let's read out the header:

In [ ]:
hdulist[0].header

Now we know a little bit about what we're working with. You'll have to check headers pretty frequently so I'd commit this step to memory!

Now we can access the data contained in the FITS file. The values are typically units of brightness and the positions in the matrix are the pixel values, with (0, 0) in the bottom left. You can manipulate this matrix like you would with any array. 

In [ ]:
img_data = hdulist[0].data

In [ ]:
plt.imshow(img_data, origin = 'lower')

# Exercises

## Exercise 1
Let's practice some more plotting skills, now incorporating units.

A. Write a function that takes an array of frequencies and spits out the Planck distribution. That's this equation:

$ B(\nu, T) = \frac{2h\nu^3/c^2}{e^{\frac{h\nu}{k_B T}} - 1}$
 
 
This requires you to use the Planck constant, the Boltzmann constant, and the speed of light from astropy. Make sure they are all in cgs.

B. Plot your function in log-log space for T = 25, 50, and 300 K. The most sensible frequency range is about $10^5$ to $10^{15}$ Hz. Hint: if your units are correct, your peak values of B(T) should be on the order of $10^{-10}$. Make sure everything is labelled.

# Exercise 2

Let's put everything together now! Here's a link to the full documentation for FITSFigure, which will tell you all of the customizable options: http://aplpy.readthedocs.io/en/stable/api/aplpy.FITSFigure.html. Let's create a nice plot of M51 with a background optical image and X-ray contours overplotted.

The data came from here if you're interested: http://chandra.harvard.edu/photo/openFITS/multiwavelength_data.html

A. Using astropy, open the X-RAY data (m51_xray.fits). Flatten the data array and find its standard deviation, and call it sigma. Use arrayname.flatten() to do this : https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html


B. Use plt.imshow() to present the image. You can find more information in the semianr 3 part II we covered.
Add contour to this image too, set levels= [3sigma], and colors = red)